In [1]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import LogFormatterSciNotation
from matplotlib.ticker import PercentFormatter
from matplotlib.ticker import LogFormatterMathtext
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap
import os
import sys
from datetime import timedelta, date
import csv
mpl.rcParams['figure.dpi'] = 300
import matplotlib.ticker as ticker

In [2]:
conn_unit_profile = sqlite3.connect("../agg-data/unit-profile.db")
df_tech = pd.read_sql_query('select * from "unit-profile"', conn_unit_profile)

#fix AT&T IPBB
df_tech['technology'] = np.where(df_tech.isp == 'AT&T IPBB', 'DSL', df_tech.technology)
df_tech['technology'] = np.where(df_tech.technology == 'IPBB', 'DSL', df_tech.technology)
df_tech['technology'] = np.where(df_tech.technology == 'UVERSE', 'DSL', df_tech.technology)

def isp_names(df, column):
    df[column] = np.where(df[column].str.startswith("Verizon"), 'Verizon', df[column])
    df[column] = np.where(df[column] == "TimeWarner", 'Time Warner Cable', df[column])
    #df[column] = np.where(df[column] == "Cablevision", 'Cablevision/Optimum', df[column])
    #df[column] = np.where(df[column] == "Optimum", 'Cablevision/Optimum', df[column])
    df[column] = np.where(df[column].str.startswith("Frontier"), 'Frontier', df[column])
    df[column] = np.where(df[column] == "Qwest", 'CenturyLink', df[column])
    df[column] = np.where(df[column].str.startswith("Cincinnati Bell"), 'Cincinnati Bell', df[column])
    df[column] = np.where(df[column].str.startswith("AT&T"), 'AT&T', df[column])
    df[column] = np.where(df[column] == "TWC", 'Time Warner Cable', df[column])
    
isp_names(df_tech, 'isp')

df_tech_for_merge = pd.DataFrame()
for year in ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']:
    #print(year)
    if year == '2012':
        df_tmp = df_tech[df_tech.year == '2012']
        df_tmp_for_append = df_tmp[df_tmp.month == '09']
        list_2012_newer_units = df_tmp_for_append.unit_id.to_list()
        list_2012_older_units = df_tmp[df_tmp.month == '04'].unit_id.to_list()
        for unit_id in list_2012_older_units:
            if not unit_id in list_2012_newer_units:
                df_tmp_for_append = pd.concat([df_tmp_for_append, df_tmp[(df_tmp.month == '04') & (df_tmp.unit_id == unit_id)]], ignore_index=True)
    elif year == '2013':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2014':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2015':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2016':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2017':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2018':
        df_tmp_for_append = df_tech[df_tech.year == '2017']
        df_tmp_for_append['year'] = '2018'
    elif year == '2019':
        df_tmp_for_append = df_tech[df_tech.year == '2017']
        df_tmp_for_append['year'] = '2019'
        
    df_tech_for_merge = pd.concat([df_tech_for_merge, df_tmp_for_append], ignore_index=True)
conn_unit_profile.close()

/u/home/fries/.local/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/u/home/fries/.local/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
df_tech_for_merge.groupby('year', as_index=False).agg({'isp':lambda x: pd.Series.unique(x).tolist()})

,year,isp
0,2012,"[Comcast, Verizon, Cablevision, AT&T, Windstre..."
1,2013,"[AT&T, Brighthouse, Cablevision, CenturyLink, ..."
2,2014,"[Time Warner Cable, Verizon, Comcast, CenturyL..."
3,2015,"[AT&T, Brighthouse, CableONE, Cablevision, Cen..."
4,2016,"[Charter, AT&T, Verizon, CenturyLink, Comcast,..."
5,2017,"[Frontier, Windstream, Cox, AT&T, CenturyLink,..."
6,2018,"[Frontier, Windstream, Cox, AT&T, CenturyLink,..."
7,2019,"[Frontier, Windstream, Cox, AT&T, CenturyLink,..."


In [12]:
sorted(list(df_tech[df_tech.year == '2018'].isp.unique()))

['AT&T',
 'CenturyLink',
 'Charter',
 'Cincinnati Bell',
 'Comcast',
 'Cox',
 'Frontier',
 'Mediacom',
 'Optimum',
 'Verizon',
 'Windstream']

In [14]:
sorted(list(df_tech_for_merge[df_tech_for_merge.year == '2018'].isp.unique()))

['AT&T',
 'CenturyLink',
 'Charter',
 'Cincinnati Bell',
 'Comcast',
 'Cox',
 'Frontier',
 'Hawaiian Telcom',
 'Hughes',
 'Mediacom',
 'Optimum',
 'Verizon',
 'Wildblue/ViaSat',
 'Windstream']

In [11]:
sorted(list(df_tech[df_tech.year == '2019'].isp.unique()))

['CenturyLink',
 'Charter',
 'Cincinnati Bell',
 'Comcast',
 'Cox',
 'Frontier',
 'Mediacom',
 'Optimum',
 'Verizon',
 'Windstream']

In [13]:
sorted(list(df_tech_for_merge[df_tech_for_merge.year == '2019'].isp.unique()))

['AT&T',
 'CenturyLink',
 'Charter',
 'Cincinnati Bell',
 'Comcast',
 'Cox',
 'Frontier',
 'Hawaiian Telcom',
 'Hughes',
 'Mediacom',
 'Optimum',
 'Verizon',
 'Wildblue/ViaSat',
 'Windstream']

In [15]:
df_satellite_isps = df_tech[(df_tech.year == '2017') & (df_tech.isp.isin(['Hughes', 'Wildblue/ViaSat']))]
satellite_unit_ids = df_satellite_isps.unit_id.unique().tolist()
for swapped_to_non_sat in df_tech[(df_tech.year == '2019') & (df_tech.unit_id.isin(satellite_unit_ids))].unit_id.unique():
    satellite_unit_ids.remove(swapped_to_non_sat)
usable_sat_units = satellite_unit_ids

#ignoring differences in download speeds;
#see http-get-mt-cdf for some analysis of differences
#> 15 Mbps, which would also exclude the units two cells below this one

#consistently has less than 500ms latency in 2019
usable_sat_units.remove(1273)
usable_sat_units.remove(801252)
usable_sat_units.remove(13236)
#likely switched ISP in 2019 because second half of the year has less than 500ms latency:
usable_sat_units.remove(999051)
usable_sat_units.remove(216686)

In [17]:
sorted(usable_sat_units)

[605,
 9328,
 10262,
 10579,
 12857,
 12893,
 31967,
 32379,
 32384,
 32385,
 32387,
 32390,
 32393,
 32398,
 32400,
 32401,
 32403,
 32406,
 32408,
 32413,
 32415,
 32416,
 32418,
 32419,
 32421,
 32424,
 32548,
 32551,
 32561,
 32564,
 32567,
 32573,
 589792,
 605674,
 614078,
 614110,
 614120,
 614134,
 614184,
 614194,
 614196,
 614220,
 615144,
 615152,
 615366,
 620744,
 620750,
 623976,
 623992,
 649218,
 657942,
 664658,
 797354,
 797700,
 797804,
 799152,
 799496,
 799514,
 801248,
 806268,
 915802,
 938998,
 939004,
 939188,
 939190,
 940672,
 940808,
 941604,
 941606,
 941608,
 941610,
 941612,
 941776,
 941934,
 941972,
 941988,
 941990,
 942240,
 942528,
 942536,
 942590,
 942596,
 942624,
 942632,
 942654,
 942658,
 942684,
 942818,
 943274,
 943290,
 945144,
 947106,
 947120,
 948120,
 948134,
 948436,
 948956,
 949030,
 949082,
 949084,
 949148,
 949150,
 949158,
 977874,
 997455,
 997481,
 997489,
 997981,
 997989,
 997993,
 997999,
 998003,
 998005,
 998015,
 998019,


In [21]:
set([605,
 9328,
 10262,
 10579,
 12857,
 12893,
 31967,
 32379,
 32384,
 32385,
 32387,
 32390,
 32393,
 32398,
 32400,
 32401,
 32403,
 32406,
 32408,
 32413,
 32415,
 32416,
 32418,
 32419,
 32421,
 32424,
 32548,
 32551,
 32561,
 32564,
 32567,
 32573,
 589792,
 605674,
 614078,
 614110,
 614120,
 614134,
 614184,
 614194,
 614196,
 614220,
 615144,
 615152,
 615366,
 620744,
 620750,
 623976,
 623992,
 649218,
 657942,
 664658,
 797354,
 797700,
 797804,
 799152,
 799496,
 799514,
 801248,
 806268,
 915802,
 938998,
 939004,
 939188,
 939190,
 940672,
 940808,
 941604,
 941606,
 941608,
 941610,
 941612,
 941776,
 941934,
 941972,
 941988,
 941990,
 942240,
 942528,
 942536,
 942590,
 942596,
 942624,
 942632,
 942654,
 942658,
 942684,
 942818,
 943274,
 943290,
 945144,
 947106,
 947120,
 948120,
 948134,
 948436,
 948956,
 949030,
 949082,
 949084,
 949148,
 949150,
 949158,
 977874,
 997455,
 997481,
 997489,
 997981,
 997989,
 997993,
 997999,
 998003,
 998005,
 998015,
 998019,
 998025,
 998027,
 998035,
 998039,
 998041,
 998043,
 998045,
 998053,
 998067,
 998331,
 998333,
 998339,
 998343,
 998345,
 998367,
 998373,
 998777,
 998783,
 998787,
 998795,
 998807,
 998997,
 998999,
 999003,
 999005,
 999011,
 999017,
 999019,
 999023,
 999805,
 999809,
 1000782,
 1006268,
 1006334,
 1006420,
 1006622,
 1006960,
 1006964,
 1009114,
 1009360,
 1009388,
 1009400,
 1009534,
 1009996,
 1010472]) - set([605,
 12893,
 13236,
 32384,
 32385,
 32403,
 32419,
 32548,
 216686,
 589792,
 605674,
 614078,
 614120,
 614134,
 614184,
 614196,
 614220,
 615144,
 615152,
 615366,
 620744,
 620750,
 623976,
 623992,
 797804,
 799496,
 801248,
 806268,
 915802,
 939188,
 939190,
 940672,
 941604,
 941610,
 941612,
 941776,
 941934,
 941972,
 941988,
 942528,
 942536,
 942596,
 942632,
 942654,
 943274,
 943290,
 945144,
 948134,
 948956,
 949030,
 949082,
 949084,
 949158,
 997489,
 997989,
 997993,
 998005,
 998015,
 998019,
 998025,
 998035,
 998039,
 998045,
 998053,
 998339,
 998343,
 998345,
 998373,
 998783,
 998807,
 998997,
 999003,
 999005,
 999017,
 999019,
 1006420,
 1006964,
 1009114,
 1009360,
 1009388,
 1009534,
 1009996])

{9328,
 10262,
 10579,
 12857,
 31967,
 32379,
 32387,
 32390,
 32393,
 32398,
 32400,
 32401,
 32406,
 32408,
 32413,
 32415,
 32416,
 32418,
 32421,
 32424,
 32551,
 32561,
 32564,
 32567,
 32573,
 614110,
 614194,
 649218,
 657942,
 664658,
 797354,
 797700,
 799152,
 799514,
 938998,
 939004,
 940808,
 941606,
 941608,
 941990,
 942240,
 942590,
 942624,
 942658,
 942684,
 942818,
 947106,
 947120,
 948120,
 948436,
 949148,
 949150,
 977874,
 997455,
 997481,
 997981,
 997999,
 998003,
 998027,
 998041,
 998043,
 998067,
 998331,
 998333,
 998367,
 998777,
 998787,
 998795,
 998999,
 999011,
 999023,
 999805,
 999809,
 1000782,
 1006268,
 1006334,
 1006622,
 1006960,
 1009400,
 1010472}

In [22]:
df_att = df_tech[(df_tech.year == '2018') & (df_tech.isp.isin(['AT&T']))]
df_att = df_att[~df_att.unit_id.isin(df_tech[df_tech.year == '2019'].unit_id.unique())]
att_probes_carry_fwd = df_att.unit_id.unique().tolist()
att_probes_carry_fwd

[507,
 563,
 790,
 1087,
 1203,
 5596,
 5616,
 5655,
 5757,
 5842,
 6129,
 6305,
 7179,
 7226,
 7416,
 7460,
 7826,
 8375,
 8807,
 8969,
 9290,
 9598,
 9616,
 9953,
 10037,
 10610,
 10793,
 10910,
 10911,
 11123,
 11366,
 11524,
 13412,
 26431,
 27698,
 27738,
 27786,
 28761,
 32549,
 184442,
 193026,
 210406,
 210450,
 215698,
 216054,
 216146,
 217190,
 605662,
 615112,
 623956,
 625876,
 625882,
 625932,
 636350,
 660294,
 669074,
 797592,
 798324,
 798724,
 805054,
 805056,
 805166,
 805270,
 805276,
 805280,
 805292,
 805296,
 805384,
 805490,
 805504,
 805538,
 805558,
 805630,
 805634,
 805760,
 805778,
 805782,
 805804,
 805874,
 805912,
 805950,
 805960,
 805976,
 805996,
 806086,
 806090,
 806096,
 806186,
 806256,
 806278,
 806282,
 806290,
 806298,
 806300,
 806330,
 811882,
 811954,
 811982,
 882106,
 915624,
 925818,
 939602,
 940766,
 941698,
 941780,
 942636,
 945120,
 948526,
 990695,
 994185,
 994195,
 994203,
 994205,
 994209,
 994221,
 994235,
 994249,
 994419,
 994

In [31]:
df_satellite_isps = df_tech[(df_tech.year == '2017') & (df_tech.isp.isin(['Hughes', 'Wildblue/ViaSat']))]
df_satellite_isps = df_satellite_isps[~df_satellite_isps.unit_id.isin(df_tech[df_tech.year.isin(['2019', '2018'])].unit_id.unique())]
usable_sat_units = df_satellite_isps.unit_id.unique().tolist()
usable_sat_units.remove(1273)
usable_sat_units.remove(801252)
usable_sat_units.remove(13236)
usable_sat_units.remove(999051)
usable_sat_units.remove(216686)


df_att = df_tech[(df_tech.year == '2018') & (df_tech.isp.isin(['AT&T']))]
df_att = df_att[~df_att.unit_id.isin(df_tech[df_tech.year == '2019'].unit_id.unique())]
att_probes_carry_fwd = df_att.unit_id.unique().tolist()
att_probes_carry_fwd.remove(507)
att_probes_carry_fwd.remove(5655)
att_probes_carry_fwd.remove(7460)
att_probes_carry_fwd.remove(27738)
att_probes_carry_fwd.remove(216054)
att_probes_carry_fwd.remove(216146)
att_probes_carry_fwd.remove(805166)
att_probes_carry_fwd.remove(805490)
att_probes_carry_fwd.remove(805630)
att_probes_carry_fwd.remove(805782)
att_probes_carry_fwd.remove(811954)
att_probes_carry_fwd.remove(941780)
att_probes_carry_fwd.remove(994195)
att_probes_carry_fwd.remove(994419)
att_probes_carry_fwd.remove(999719)

#Hawaiian Telcom
df_hitel = df_tech[(df_tech.year == '2017') & (df_tech.isp.isin(['Hawaiian Telcom']))]
df_hitel = df_hitel[~df_hitel.unit_id.isin(df_tech[df_tech.year.isin(['2019', '2018'])].unit_id.unique())]
hit_probes_carry_fwd = df_hitel.unit_id.unique().tolist()
hit_probes_carry_fwd.remove(995125)
hit_probes_carry_fwd.remove(995121)

df_tech_for_merge = pd.DataFrame()
for year in ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']:
    #print(year)
    if year == '2012':
        df_tmp = df_tech[df_tech.year == '2012']
        df_tmp_for_append = df_tmp[df_tmp.month == '09']
        list_2012_newer_units = df_tmp_for_append.unit_id.to_list()
        list_2012_older_units = df_tmp[df_tmp.month == '04'].unit_id.to_list()
        for unit_id in list_2012_older_units:
            if not unit_id in list_2012_newer_units:
                df_tmp_for_append = pd.concat([df_tmp_for_append, df_tmp[(df_tmp.month == '04') & (df_tmp.unit_id == unit_id)]], ignore_index=True)
    elif year == '2013':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2014':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2015':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2016':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2017':
        df_tmp_for_append = df_tech[df_tech.year == year]
    elif year == '2018':
        df_tmp_for_append = df_tech[(df_tech.year == '2017') & (df_tech.unit_id.isin(usable_sat_units + hit_probes_carry_fwd))]
        df_tmp_for_append['year'] = '2018'
        print(len(df_tmp_for_append))
        df_tmp_for_append = pd.concat([df_tmp_for_append, df_tech[df_tech.year == year]], ignore_index=True)
    elif year == '2019':
        df_tmp_for_append = df_tech[(df_tech.year == '2017') & (df_tech.unit_id.isin(usable_sat_units + hit_probes_carry_fwd))]
        df_tmp_for_append = pd.concat([df_tmp_for_append, df_tech[(df_tech.year == '2017') & (df_tech.unit_id.isin(att_probes_carry_fwd))]], ignore_index=True)
        df_tmp_for_append['year'] = '2019'
        print(len(df_tmp_for_append))
        df_tmp_for_append = pd.concat([df_tmp_for_append, df_tech[df_tech.year == year]], ignore_index=True)
        
    df_tech_for_merge = pd.concat([df_tech_for_merge, df_tmp_for_append], ignore_index=True)

211
355


/u/home/fries/.local/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [27]:
sorted(list(df_tech_for_merge[df_tech_for_merge.year == '2018'].isp.unique()))

['AT&T',
 'CenturyLink',
 'Charter',
 'Cincinnati Bell',
 'Comcast',
 'Cox',
 'Frontier',
 'Hughes',
 'Mediacom',
 'Optimum',
 'Verizon',
 'Wildblue/ViaSat',
 'Windstream']

In [28]:
sorted(list(df_tech_for_merge[df_tech_for_merge.year == '2019'].isp.unique()))

['AT&T',
 'CenturyLink',
 'Charter',
 'Cincinnati Bell',
 'Comcast',
 'Cox',
 'Frontier',
 'Hughes',
 'Mediacom',
 'Optimum',
 'Verizon',
 'Wildblue/ViaSat',
 'Windstream']